In [ ]:
tweepy.__version__

'3.10.0'

In [ ]:
!pip install tweepy==4.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tweepy
#from twitter_authentication import bearer_token
import time
import pandas as pd

In [ ]:
client = tweepy.Client('AAAAAAAAAAAAAAAAAAAAAONwiAEAAAAAywolbunD%2F%2B6Eb78%2Bj%2FpnSxIz5mA%3DglS08ahSE89Uipor00aJBVnABE85UZtHkiCvuiuexGEpAjrw3O', wait_on_rate_limit=True)

In [ ]:
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'point_radius:[-120.6596 35.2828 25km] has:geo' ,
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2021-01-20T00:00:00Z',
                                 end_time = '2021-01-21T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    hoax_tweets.append(response)

In [ ]:
hoax_tweets[0].data[0]

<Tweet id=1352042184774660096 text=Just posted a photo @ Morro Bay, California https://t.co/jpSOzVBcV7>

In [ ]:
result = []
user_dict = {}
# Loop through each response object
for response in hoax_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'geo' : tweet.geo,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [ ]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,text,geo,created_at,retweets,replies,likes,quote_count
0,35800299,GIOVANNISFISH,639,3160,Giovanni’s Fish Market is Morro Bay’s oldest f...,"Morro Bay, CA","Just posted a photo @ Morro Bay, California ht...","{'coordinates': {'type': 'Point', 'coordinates...",2021-01-20 23:55:51+00:00,0,0,0,0
1,4471843394,hebrewhustla,300,33312,🌈she/her | just be cool,"San Francisco, CA",a weird comparison of what we did next: I took...,{'place_id': '057f8a6fa3c286f9'},2021-01-20 23:44:31+00:00,0,0,1,0
2,819732579104550913,keltz_k,71,5183,“Those who would give up liberty to purchase a...,California,@sputnikwriter @tobyzakaria @mawilner @fran_ch...,{'place_id': '011b3d245e51de7a'},2021-01-20 23:44:15+00:00,0,0,0,0
3,716531447255982080,nicholas_coffey,407,3124,34 year old US citizen. Libertarian. Pescatari...,"La Crosse, Wisconsin",@michaelmalice ... What?,{'place_id': '057f8a6fa3c286f9'},2021-01-20 23:39:03+00:00,0,0,0,0
4,235637573,805DarrelleR,454,32020,California country girl! Disabled College stud...,"Atascadero, CA",@DaaChazz I would like dinner but no money for...,{'place_id': '2bd8aff9ffdbc99a'},2021-01-20 23:34:29+00:00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,235637573,805DarrelleR,454,32020,California country girl! Disabled College stud...,"Atascadero, CA",@GridGamingIO @byeagle37 @BetOnDrew #FreeMacBook,{'place_id': '2bd8aff9ffdbc99a'},2021-01-20 00:17:19+00:00,0,0,1,0
307,235637573,805DarrelleR,454,32020,California country girl! Disabled College stud...,"Atascadero, CA",@Stephen96502560 @RotoRadarCEO I’m good my col...,{'place_id': '2bd8aff9ffdbc99a'},2021-01-20 00:16:03+00:00,0,1,1,0
308,983892348,Marinabeannn,401,9191,Phil 4:13 + Cal Poly SLO Rodeo💗,"San Luis Obispo, CA",you made my best friend cry to drivers license...,{'place_id': '057f8a6fa3c286f9'},2021-01-20 00:14:16+00:00,0,0,6,0
309,1327031596835332097,justredjeep,2,19,Proud jeep owner,None,"Just posted a photo @ Pismo Beach, California ...","{'coordinates': {'type': 'Point', 'coordinates...",2021-01-20 00:13:34+00:00,0,0,1,0


In [ ]:
df['geo'][7]

{'place_id': '2bd8aff9ffdbc99a'}